
Talia

Congruence closure Leo

GATs

Bildingmaier

https://arxiv.org/pdf/2505.08496  Weighted Rewriting: Semiring Semantics for
Abstract Reduction Systems
Provenance / semiring semantics is an interesting variant of proof objects. Having proof objects for ring like things makes proof theory truncate naturally (free resolution)
Scallop. Max plus.


Datalog for tensor equations. Let the compiler do the rearrangement. Linear Relations. Born approximation. Dyson. 





In [22]:
from kdrag.solvers.egraph import EGraph
from kdrag.all import *
from dataclasses import dataclass
@dataclass
class DEGraph(EGraph):
    types : dict[int, set[int]]
    def __init__(self):
        super().__init__()
        self.types = {}
    def add_type(self, S : smt.ExprRef):
        # get held sort
        assert smt.is_array(S)
        self.add_term(S)
        Sid = self.find(S)
        if Sid not in self.types:
            tids = set()
            print(S)
            T = S.domain()
            for tid in self.roots[T]:
                with self.solver:
                    self.solver.add(smt.Not(S[self.terms[tid]]))
                    if self.solver.check() == smt.unsat:
                        tids.add(tid)
            self.types[Sid] = tids
    def assign_type(self, t, A):
        tid, Aid = self.find(t), self.find(A)
        self.types[Aid].add(tid)
        





E = DEGraph()
Vec = smt.Function("Vec", smt.IntSort(), smt.SetSort(smt.SeqSort(smt.BoolSort())))
Vec(3).sort().domain()
#n = smt.Int("n")
#l = smt.
Fin = smt.Function("Fin", smt.IntSort(), smt.SetSort(smt.IntSort()))
E.add_type(Fin(0))
E.add_type(Fin(1))
E.assign_type(smt.IntVal(0), Fin(1))
E


Fin(0)
Fin(1)


DEGraph(roots=defaultdict(<class 'set'>, {Array(Int, Bool): {17, 39}, Int: {16, 35}}), terms={17: Fin(0), 16: 0, 39: Fin(1), 35: 1}, uf={}, solver=[], reasons={}, types={17: set(), 39: {16}})

In [1]:
from kdrag.solvers.egraph import EGraph
from kdrag.all import *
class DependentEGraph(EGraph):
    types : dict[smt.ExprRef, set[smt.ExprRef]]
    def is_type(self, A: smt.ExprRef) -> bool:
        return self.find(A) in self.sets
    def make_type(self, expr: smt.ExprRef) -> None:
        if expr not in self.sets:
            self.sets[expr] = set()
    def has_type(self, t : smt.ExprRef, A : smt.ExprRef) -> bool:
        A = self.find(A)
        if self.is_type(A):
            return self.find(t) in self.sets[A]
        else:
            return False
    def assign_type(self, t: smt.ExprRef, A: smt.ExprRef) -> None:
        A = self.find(A)
        if not self.is_type(A):
            raise ValueError(f"Cannot assign type {A} to {t}, {A} is not a type")
        t = self.find(t)
        self.sets[A].add(t)
    def is_def_eq(self, t1: smt.ExprRef, t2: smt.ExprRef, A : smt.ExprRef) -> bool:
        return self.has_type(t1, A) and self.has_type(t2, A) and self.find(t1).eq(self.find(t2))
    def assert_def_eq(self, t1: smt.ExprRef, t2: smt.ExprRef, A: smt.ExprRef) -> None:
        A = self.find(A)
        self.assign_type(t1, A)
        self.assign_type(t2, A)
        self.union(t1, t2)
    def rebuild(self):
        super().rebuild()
        newsets = defaultdict(set)
        for k, v in self.sets.items():
            newsets[self.find(k)] |= {self.find(x) for x in v}
        self.sets = newsets